In [1]:
import pandas as pd

log = pd.read_csv('visit_log.csv', sep=';')

log.loc[(log.traffic_source == 'yandex') | (log.traffic_source == 'google'), 'source_type'] = 'organic'
log.loc[((log.traffic_source == 'paid') | (log.traffic_source == 'email')) & (log.region == 'Russia'), 'source_type'] = 'ad'
log.loc[((log.traffic_source == 'paid') | (log.traffic_source == 'email')) & (log.region != 'Russia'), 'source_type'] = 'other'
log.loc[log.source_type.isnull(), 'source_type'] = log.traffic_source

log.head(15)

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic
5,1549980742,8855508aad,https://host.ru/df646c3676cc259fa0,Russia,fc43898e47,yandex,organic
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid,ad
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,Russia,cb5082b6f6,direct,direct
8,1549980760,af5570f5a1,https://host.ru/3004a8273caeef2867,China,45664f7af2,direct,direct
9,1549980765,3e7077fd2f,https://host.ru/df646c3676cc259fa0,Russia,6f9de8c8b6,email,ad


In [2]:
import re

news = pd.read_csv('URLs.txt')

news[news.url.str.contains(r'/\w+/\d{8}-')].head(15)

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...
10,/middleeast/36131117-divizion-s-400-ne-zametil...
11,/economics/36065674-rossiyane-vozmutilis-minim...
12,/politics/36118047-otets-24-detey-potreboval-i...
13,/travel/36194479-v-sheremetevo-passazhiry-ustr...
14,/politics/35638742-nesolidno-mariya-zaharova-s...


In [3]:
ratings = pd.read_csv('ratings.csv')

#отбираем пользователей, выставивших > 100 оценок
ratings_over = ratings.groupby('userId').count().query('rating > 100').reset_index()[['userId']]
#в dataframe rating путем merge оставляем только тех пользователей, которые получилить на предыдущем шаге
ratings_merged = ratings_over.merge(ratings, on='userId', how='inner')
#определяем максимальное и минимальное значение timestamp для каждого пользователя
agg_res = ratings_merged.groupby('userId').agg(['max', 'min'])['timestamp']
#подсчитываем разницу между максимальным и минимальным значением timestamp для каждого пользователя
agg_res['diff'] = agg_res['max'] - agg_res['min']
#определяем среднее значение разницы среди полученных на предыдущем шаге значений
agg_res['diff'].mean()

40080507.4496124

In [4]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

earnings_rzd = client_base.merge(rzd, on='client_id', how='outer')[['client_id', 'rzd_revenue']]
earnings_auto = earnings_rzd.merge(auto, on='client_id', how='outer')
#таблица без адреса
earnings_total = earnings_auto.merge(air, on='client_id', how='outer')
#таблица с адресом
earnings_total_addr = earnings_total.merge(client_base, on='client_id', how='outer')

earnings_total_addr.head(10)

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4
